In [ ]:
import matplotlib, os, random
import numpy as np
import imageio.v3 as imageio
from IPython.display import clear_output
from shared.Render import collect_saved_frames, render_TSP
from shared.TestFunctions import TEST_FUNCTIONS
matplotlib.use('Agg')
np.random.seed(42)

#### Helper functions for the genetic algorithm

In [3]:
def evalueate_TSP_path(ids: list, positions: np.ndarray):
    distance = 0
    for idx in len(ids) - 1:
        id1, id2 = ids[idx], ids[idx] + 1
        diff_x = positions[id1][0] - positions[id2][0]
        diff_y = positions[id1][1] - positions[id2][1]
        distance += np.sqrt(diff_x + diff_y)
    id1, id2 = ids[-1], ids[0]
    diff_x = positions[id1][0] - positions[id2][0]
    diff_y = positions[id1][1] - positions[id2][1]
    distance += np.sqrt(diff_x + diff_y)
    return distance

In [ ]:
def crossover(parent_A: list, parent_B: list):
    offspring_AB = parent_A[:(len(parent_A) + 1) // 2]
    selected = dict(offspring_AB)
    for id in parent_B:
        if id not in selected:
            offspring_AB.append(id)
    return offspring_AB

In [ ]:
def mutate(offspring_AB: list):
    a, b = random.sample(range(len(offspring_AB)), 2)
    offspring_AB[a], offspring_AB[b] = offspring_AB[b], offspring_AB[a]
    return offspring_AB

#### The genetic algorithm for TSP problem

In [ ]:
def genetic_algorithm(
        positions: np.ndarray,
        num_individuals: int = 20,
        num_iter: int = 200):
    population = [np.random.permutation(20) for _ in range(num_individuals)]
    values = [evalueate_TSP_path(individual, positions) for individual in population]
    for parent_idx in range(num_individuals):
        yield None, parent_idx, None, population[parent_idx], values[parent_idx], False
    for iter in range(num_iter):
        new_population = population.copy()
        new_values = values.copy()
        for parent_A_idx in range(num_individuals):
            parent_B_idx = random.sample(range(num_individuals), 1)
            offspring_AB = crossover(population[parent_A_idx], population[parent_B_idx])
            mutated = False
            if np.random.uniform() < 0.5:
                offspring_AB = mutate(offspring_AB)
                mutated = True
            offspring_value = evalueate_TSP_path(offspring_AB, positions)
            if offspring_value < values[parent_A_idx]:
                new_population[parent_A_idx] = offspring_AB
                new_values[new_values] = offspring_value
                yield iter, parent_A_idx, parent_B_idx, offspring_AB, offspring_value, mutated
        population = new_population

#### Generate random positions on 2D plane

In [6]:
num_positions = 20
positions_range = (-20, 20)
positions = np.random.uniform(positions_range[0], positions_range[1], size=(num_positions, 2))

In [2]:
pts = [(3.2, 5.2, 1), (1.7, 2.2, 3), (4.1, 3.6, 2)]
frame = render_TSP(pts, title="Demo: Points connected by arrows")
os.mkdir("gifs/genetic")
imageio.imwrite(os.path.join("gifs/genetic", f"genetic{1:08d}.png"), frame)